In [6]:
!pip install settrade

You should consider upgrading via the 'c:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [7]:
import settrade.openapi
import pandas as pd
#import datetime
import pytz
from datetime import datetime, timedelta

In [18]:
tz=pytz.timezone("Asia/Bangkok")

In [41]:
#########################Login Part###########################
 
app_id="hBmEmp5KTnGaoF7V"                                 # Your app ID
app_secret="YXSg2ee0+YYPv9GeTmWADtuv6rzyD5pzmwXPtVZdzBA=" # Your app Secret
broker_id="063"
app_code="ALGO"
derv_account_no="0033997" #settrade-D
eq_account_no="0033997-E" # settrade-E
is_auto_queue = False
 
investor = settrade.openapi.Investor(app_id,app_secret,app_code,broker_id,is_auto_queue)

In [42]:
acc=investor.Derivatives(account_no=derv_account_no)
acc.get_account_info()

{'success': False,
 'status_code': 403,
 'data': {},
 'message': '{"additionalInfo":{"message":"Unexpected seosdgateway error with http status: 403"},"message":"Unexpected seosdgateway error with http status: 403","code":"API-403"}'}

In [21]:
portfolio=acc.get_portfolio()
portfolio

{'success': True, 'status_code': 200, 'data': [], 'message': ''}

In [22]:
acc_eq=investor.Equity(account_no=eq_account_no)
acc_eq.get_account_info()

{'success': False,
 'data': {},
 'message': 'Internal Error: Get https://open-api.settrade.com/api/seos/v1/063/accounts/0033997-E/account-info: dial tcp: lookup open-api.settrade.com: no such host',
 'status_code': 500}

In [23]:
 
##################################
# กรณี Investor
mqtt = investor.MQTTWebsocket()
market = investor.MarketData()
 
###################################

In [24]:
instrument="S50Z20C1000"

In [ ]:
instrument="S50H21"

In [26]:
def Period(interval):
   switcher={
       "1m": "M",
       "5m": "M",
       "15m": "M",
       "30m": "M",
       "60m": "H",
       "240m": "H",
       "1d": "D"
   }
   return switcher[interval]

In [27]:
############################
fmt="%Y-%m-%dT%H:%M:%S"
interval="1d"
interval="5m"
interval="15m"
interval="30m"
interval="60m"
interval="1m"
limit=1000
 

In [39]:

now=datetime.now()
if Period(interval)=="M":
  time_back=timedelta(minutes=limit)
elif Period(interval)=="H":
  time_back=timedelta(hours=limit)
elif Period(interval)=="D":
  time_back=timedelta(days=limit)
else:
  time_back=timedelta(days=limit)
  
start=now - time_back
start=datetime.strftime(start,fmt)
end=datetime.strftime(now,fmt)
 
#print(start)
normalize=True
instrument="S50H21"
ChartDataResponse = market.get_candlestick(instrument,interval,limit,start,end,normalize)

In [40]:
df=pd.DataFrame(ChartDataResponse["data"])
df["datetime"]=df["time"].apply(lambda x: datetime.fromtimestamp(x, tz=tz) )
col=["datetime","open","high","low","close","volume"]
df[col].tail()

,datetime,open,high,low,close,volume


In [36]:
df.describe()

ValueError: Cannot describe a DataFrame without columns

In [ ]:
last_price=0.0
def price_info_message(result, subscriber, arg1, arg2):
  global last_price
  print("price_info_message: ", result)
  print("price_info_message:  instrument: ", arg1 , arg2)
  last_price = result['data']['last']
  print('last price is', last_price)
    #if last_price > 100:
    #print("stop subscribe")
        #subscriber.stop()

sub_price_info = mqtt.subscribe_price_info(instrument, on_message = price_info_message , args=(instrument, "xxx" )  )
sub_price_info.start()


price_info_message:  {'is_success': True, 'data': {'symbol': 'S50H21', 'high': 934.0, 'low': 926.9, 'last': 933.2, 'total_volume': 42259, 'projected_open_price': 0.0}, 'message': ''}
price_info_message:  instrument:  S50H21 xxx
last price is 933.2


In [ ]:
last_price

933.2

In [ ]:
sub_price_info.stop()

In [ ]:
def candle_message(result, subscriber, arg1):
  print("candle_message: ", result)
    #if num >10:
    #  subscriber.stop()
 
 
sub_candle = mqtt.subscribe_candlestick(instrument,interval, candle_message,args=(instrument,))
sub_candle.start()

In [ ]:
sub_candle.stop()

In [ ]:
bid_price1=0.0
ask_price1=0.0
bid_volume1=0.0
ask_volume1=0.0

def bid_offer_message(result, subscriber, arg1):
  global bid_price1, ask_price1, bid_volume1,  ask_volume1
  print("bid_offer_message: ", result)
  #print(num)
  bid_price1 = result['data']['bid_price1']
  ask_price1 = result['data']['ask_price1']
  bid_volume1 = result['data']['bid_volume1']
  ask_volume1 = result['data']['ask_volume1']
  print('bid price 1 price is', bid_price1)
  #if bid_price1 > 100:
    #print("stop subscribe")
    #subscriber.stop()
 
sub_bidoffer = mqtt.subscribe_bid_offer( instrument, on_message = bid_offer_message ,args=(instrument,))
sub_bidoffer.start()

bid_offer_message:  {'is_success': True, 'data': {'symbol': 'S50H21', 'bid_price1': 933.1, 'bid_price2': 933.0, 'bid_price3': 932.9, 'bid_price4': 932.8, 'bid_price5': 932.7, 'ask_price1': 933.1, 'ask_price2': 933.0, 'ask_price3': 932.9, 'ask_price4': 932.8, 'ask_price5': 932.7, 'bid_volume1': 10, 'bid_volume2': 339, 'bid_volume3': 479, 'bid_volume4': 589, 'bid_volume5': 305, 'ask_volume1': 70, 'ask_volume2': 322, 'ask_volume3': 554, 'ask_volume4': 392, 'ask_volume5': 121}, 'message': ''}
bid price 1 price is 933.1


In [ ]:
print("bid: ", bid_price1, "vol bid: ",  bid_volume1,  "\nask: ", ask_price1, "vol ask: ", ask_volume1)

bid:  933.1 vol bid:  10 
ask:  933.1 vol ask:  70


In [ ]:
sub_bidoffer.stop()

In [ ]:
def derv_order_message(result, subscriber, arg1):
  print("derv_order_message: ", result)
 
sub_order = mqtt.subscribe_derivatives_order(derv_account_no, on_message = derv_order_message , args=(derv_account_no,))
sub_order.start()

In [ ]:
sub_order.stop()

In [ ]:
place_order = acc.place_order(
    symbol=instrument,
    price=ask_price1,
    volume=10,
    side="LONG",
    position="OPEN",
    pin="000000")

place_order

In [ ]:
place_order

candle_message:  {'is_success': True, 'data': {'symbol': 'S50H21', 'interval': '1m', 'last_sequence': 343493, 'time': 1608108480, 'open': 947.6, 'high': 948.0, 'low': 947.6, 'close': 948.0, 'volume': 4}, 'message': ''}


{'data': {'order_no': 2172},
 'message': '',
 'status_code': 200,
 'success': True}

In [ ]:
 trade_list=acc.get_trades()
 trade_list

{'data': [], 'message': '', 'status_code': 200, 'success': True}

In [ ]:
portfolio=acc.get_portfolio()
portfolio

candle_message:  

{'data': [], 'message': '', 'status_code': 200, 'success': True}

{'is_success': True, 'data': {'symbol': 'S50H21', 'interval': '1m', 'last_sequence': 352470, 'time': 1608109020, 'open': 947.2, 'high': 947.4, 'low': 947.2, 'close': 947.2, 'volume': 7}, 'message': ''}


In [ ]:
acc.get_orders()